In [0]:
#один произвольный запрос на соединение двух наборов данных
#один произвольный запрос на группировку набора данных с использованием функций агрегирования
import pandas as pd
import time
import numpy as np

In [49]:
! pip install -U pandasql

import pandasql as pdsql

Requirement already up-to-date: pandasql in /usr/local/lib/python3.6/dist-packages (0.7.3)


In [50]:
url = 'https://raw.githubusercontent.com/NastjaK666/uni/master/winequality-red.csv'

data = pd.read_csv(url)
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [52]:
# Запрос с агрегацией в Pandas

aggregations = {
    'pH':{
        'min_pH':'min',
        'max_pH':'max',
        'average_pH':'mean'
    },
    'density':{
        'min_density':'min',
        'max_density':'max',
        'average_density':'mean'
    },
            
    'alcohol':{
        'min_alcohol':'min',
        'max_alcohol':'max',
        'average_alcohol':'mean'
    },
                    
                    

    'quality':{ 'number_of_samples':'count'}

    
}

data.groupby('quality').agg(aggregations)

/usr/local/lib/python3.6/dist-packages/pandas/core/groupby/generic.py:1315: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


pH                    ...     alcohol                           quality
        min_pH max_pH average_pH  ... max_alcohol average_alcohol number_of_samples
quality                           ...                                              
3         3.16   3.63   3.398000  ...        11.0        9.955000                10
4         2.74   3.90   3.381509  ...        13.1       10.265094                53
5         2.88   3.74   3.304949  ...        14.9        9.899706               681
6         2.86   4.01   3.318072  ...        14.0       10.629519               638
7         2.92   3.78   3.290754  ...        14.0       11.465913               199
8         2.88   3.72   3.267222  ...        14.0       12.094444                18

[6 rows x 10 columns]

In [54]:
SQLQuerly1='''
SELECT
quality,
min(pH) as min_pH,
max(pH) as max_pH,
avg(pH) as average_pH,

min(density) as min_density,
max(density) as max_density,
avg(density) as average_density,

min(alcohol) as min_alcohol,
max(alcohol) as max_alcohol,
avg(alcohol) as average_alcohol,

count(quality) as number_of_samples


FROM data
GROUP BY quality

'''
pdsql.sqldf(SQLQuerly1, locals())


,quality,min_pH,max_pH,average_pH,min_density,max_density,average_density,min_alcohol,max_alcohol,average_alcohol,number_of_samples
0,3,3.16,3.63,3.398000,0.99471,1.00080,0.997464,8.4,11.0,9.955000,10
1,4,2.74,3.90,3.381509,0.99340,1.00100,0.996542,9.0,13.1,10.265094,53
2,5,2.88,3.74,3.304949,0.99256,1.00315,0.997104,8.5,14.9,9.899706,681
3,6,2.86,4.01,3.318072,0.99007,1.00369,0.996615,8.4,14.0,10.629519,638
4,7,2.92,3.78,3.290754,0.99064,1.00320,0.996104,9.2,14.0,11.465913,199
5,8,2.88,3.72,3.267222,0.99080,0.99880,0.995212,9.8,14.0,12.094444,18


In [0]:
# Создание датасетов для соединения при помощи JOIN

dataframe1={
    'BookID':[0,1,2,3,4,5,6,7,8,9],
    'BookName':['Red Hood','The Witch','Monsters','Deep sea','Undertaker', 'Who is here','Forever', 'Yensen', 'Empire', 'Gone'],
    'AuthorID':[0,1,0,3,2,3,4,1,1,2],
    'ShopID':[0,1,0,1,2,1,0,0,1,1]
}
dataframe2 = {
    'AuthorID':[0,1,2,3,4],
    'AuthorName':['Will','Braun','Hotch','Daniels','Siff']
}
dataframe3 = {
    'ShopID':[0,1,2],
    'ShopName':['BiblioGlobus','BoomBook','Litra']    
}

Books = pd.DataFrame(dataframe1, columns = ['BookID','BookName','AuthorID','ShopID'])
Authors = pd.DataFrame(dataframe2, columns = ['AuthorID', 'AuthorName'])
Shops = pd.DataFrame(dataframe3, columns = ['ShopID', 'ShopName'])

In [58]:
Books.head()

,BookID,BookName,AuthorID,ShopID
0,0,Red Hood,0,0
1,1,The Witch,1,1
2,2,Monsters,0,0
3,3,Deep sea,3,1
4,4,Undertaker,2,2


In [59]:
# Запрос с соединением двух таблиц в Pandas
pd.merge(pd.merge(Books, Authors[['AuthorID', 'AuthorName']],on='AuthorID'),Shops[['ShopID','ShopName']], on='ShopID')

,BookID,BookName,AuthorID,ShopID,AuthorName,ShopName
0,0,Red Hood,0,0,Will,BiblioGlobus
1,2,Monsters,0,0,Will,BiblioGlobus
2,7,Yensen,1,0,Braun,BiblioGlobus
3,6,Forever,4,0,Siff,BiblioGlobus
4,1,The Witch,1,1,Braun,BoomBook
5,8,Empire,1,1,Braun,BoomBook
6,3,Deep sea,3,1,Daniels,BoomBook
7,5,Who is here,3,1,Daniels,BoomBook
8,9,Gone,2,1,Hotch,BoomBook
9,4,Undertaker,2,2,Hotch,Litra


In [60]:
# Запрос с соединением двух таблиц в PandaSQL
SQLQuerly2 = '''
    SELECT BookName, AuthorName, ShopName
    from Books
    join Authors
    on Books.AuthorID = Authors.AuthorID
    join Shops
    on Books.ShopID = Shops.ShopID
'''
pdsql.sqldf(SQLQuerly2, locals())

,BookName,AuthorName,ShopName
0,Red Hood,Will,BiblioGlobus
1,The Witch,Braun,BoomBook
2,Monsters,Will,BiblioGlobus
3,Deep sea,Daniels,BoomBook
4,Undertaker,Hotch,Litra
5,Who is here,Daniels,BoomBook
6,Forever,Siff,BiblioGlobus
7,Yensen,Braun,BiblioGlobus
8,Empire,Braun,BoomBook
9,Gone,Hotch,BoomBook


In [55]:
# Сравнение по времени выполнения запросов
PandasStartTime1 = time.time()
data.groupby('quality').agg(aggregations)
PandasStopTime1 = time.time()
PandaSQLStartTime1 = time.time()
pdsql.sqldf(SQLQuerly1, locals())
PandaSQLStopTime1 = time.time()
print('Время работы Pandas с запрсом с агрегацией:',PandasStopTime1 - PandasStartTime1, 'c')
print('Время работы PandaSQL с запрсом с агрегацией:',PandaSQLStopTime1 - PandaSQLStartTime1, 'c')
if PandasStopTime1 - PandasStartTime1 < PandaSQLStopTime1 - PandaSQLStartTime1:
    print('Pandas выполнил запрос быстрее на ', (PandaSQLStopTime1 - PandaSQLStartTime1) - (PandasStopTime1 - PandasStartTime1), 'c')
elif PandasStopTime1 - PandasStartTime1 < PandaSQLStopTime1 - PandaSQLStartTime1:
    print('PandaSQL выполнил запрос быстрее на ', (PandasStopTime1 - PandasStartTime1) - (PandaSQLStopTime1 - PandaSQLStartTime1), 'c')
elif PandasStopTime1 - PandasStartTime1 == PandaSQLStopTime1 - PandaSQLStartTime1:
    print('Время выполнения запроса совпало')

Время работы Pandas с запрсом с агрегацией: 0.022310733795166016 c
Время работы PandaSQL с запрсом с агрегацией: 0.028368711471557617 c
Pandas выполнил запрос быстрее на  0.0060579776763916016 c


/usr/local/lib/python3.6/dist-packages/pandas/core/groupby/generic.py:1315: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)
